# Tutorial of DiffCo for Trajectory Optimization

## 1. Create Environment for Geometrical Collision Checking

### Provide a customized class that provides the following functions:
* sample random configuration of the robot
* check if a configuration is in collision with any obstacles in the environment
* (optional) plot the robot and all objects in the environment
  
The base class CollisionEnv located at `envs/collision_env.py` provides a reference of what functions the class should provide.
```python
class CollisionEnv:
    '''
    A template class for collision environment.
    Use as a reference for implementing your own collision environment.
    '''
    def __init__(self):
        pass
    def is_collision(self, qs):
        return [self._single_collision(q) for q in qs]
    def _single_collision(self, q):
        raise NotImplementedError
    def distance(self, qs):
        return [self._single_distance(q) for q in qs]
    def _single_distance(self, q):
        raise NotImplementedError
    def sample_q(self):
        raise NotImplementedError
    def plot(self, qs):
        raise NotImplementedError
```

In this tutorial, we are going to use an environment using roboticstoolbox-python as backend, because it supports collision checking + visualization in notebook. The robot is a Franka Emika Panda robot, and the obstacles are 3 cylinders. 

*Note: the RTB simulator does not always run correctly in Colab or a notebook on a remote server.*

In [1]:
from envs.rtb import PandaThreeCylinderEnv, PandaSingleCylinderEnv

cenv = PandaThreeCylinderEnv(launch_args={'realtime': True, 'browser': 'colab'})
# cenv = PandaSingleCylinderEnv(launch_args={'realtime': True, 'browser': 'colab'})

# you may also use browser='firefox' or 'google-chrome'


Could not open specified browser, using default instead



In [2]:
print(cenv.robot.qlim)

[[-2.8973 -1.7628 -2.8973 -3.0718 -2.8973 -0.0175 -2.8973]
 [ 2.8973  1.7628  2.8973 -0.0698  2.8973  3.7525  2.8973]]


## 2. Generate Dataset for Training DiffCo

In [3]:
import torch
import numpy as np
from diffco.routines import train_test_split

num_samples = 12000
num_train = 10000
try:
    dataset = torch.load('tutorial_dataset.pt')
    train_cfgs = dataset['train_cfgs']
    test_cfgs = dataset['test_cfgs']
    train_labels = dataset['train_labels']
    test_labels = dataset['test_labels']
    cfgs = torch.cat([train_cfgs, test_cfgs])
    labels = torch.cat([train_labels, test_labels])
except FileNotFoundError:
    cfgs = torch.tensor(np.stack([cenv.sample_q() for _ in range(num_samples)]), dtype=torch.float32)
    labels = torch.tensor(cenv.is_collision(cfgs.numpy()), dtype=torch.float32)
    labels = 2 * labels - 1

    train_indices, test_indices = train_test_split(num_samples, num_train)
    train_cfgs, test_cfgs = cfgs[train_indices], cfgs[test_indices]
    train_labels, test_labels = labels[train_indices], labels[test_indices]

    torch.save({
        'train_cfgs': train_cfgs,
        'test_cfgs': test_cfgs,
        'train_labels': train_labels,
        'test_labels': test_labels,
    }, 'tutorial_dataset.pt')

print(f'cfgs.shape = {cfgs.shape}')
print(f'labels.shape = {labels.shape}')
print(f'train_cfgs.shape = {train_cfgs.shape}')
print(f'test_cfgs.shape = {test_cfgs.shape}')
print(f'train_labels.shape = {train_labels.shape}')
print(f'test_labels.shape = {test_labels.shape}')

ROS-related imports failed. This is expected if not running in a ROS environment. Otherwise, try source your ROS setup.bash file or check your ROS installation.
cfgs.shape = torch.Size([12000, 7])
labels.shape = torch.Size([12000])
train_cfgs.shape = torch.Size([10000, 7])
test_cfgs.shape = torch.Size([2000, 7])
train_labels.shape = torch.Size([10000])
test_labels.shape = torch.Size([2000])


## 3. Train DiffCo

### Provide a PyTorch FK function
that calculates a set of predefined points on your robot given a configuration. The points can be any unique points on the robot that covers all links nicely. A common choice is the origins of link frames. The function `fkine(q) -> tensor(m, d)` maps a configuration to a set of points in the d-dimensional workspace (usually, d=2 or 3)
* This is optional, but it allows us to use the FK function in the kernel, which results in much better accuracy
* This may be automatically generated in the future with a URDF file of the robot using, e.g., [differentiable-robot-model](https://github.com/facebookresearch/differentiable-robot-model) from facebookresearch 

### 3.1 Method I: using the lower-level kernel perceptron API

In [4]:
# define the pytorch FK function. 
from diffco import model, kernel
drobot = model.PandaFK()
fkine = drobot.fkine # Could be None if you don't need it

# specify kernel function for training
from diffco.kernel_perceptrons import DiffCo
fkdc = DiffCo(kernel_func='rq', transform=fkine, gamma=10)

# train the kernel perceptron
fkdc.train(train_cfgs, train_labels, max_iteration=num_train, verbose=True)

# fit the support configurations and labels with polyharmonic kernel
inference_kernel_func = kernel.Polyharmonic(k=1, epsilon=1)
fkdc.fit_poly(kernel_func=inference_kernel_func, target='label')
col_func = fkdc.poly_score

DiffCo training...


  8% 848/10000 [00:00<00:04, 2256.08it/s]


Ended at iteration 848, cost 0.3810 secs
ACC: 1.0
DiffCo training done. 0.5140 secs cost


In [5]:
# Test a random configuration
col_func(torch.tensor(cenv.sample_q(), dtype=torch.float32))

tensor([[-1.3714]])

In [6]:
# Choose a safety margin. 
# A good choice is to set it to be proportional to the minimum score.
min_score = torch.min(col_func(train_cfgs)).item()
safety_margin = min_score / 10
print(f'safety_margin = {safety_margin:.4f}')

# test the correctness of the kernel perceptron
# Ideally, TPR should be > 0.9 and ACC should be > 0.9
num_test = num_samples - num_train
test_preds = (col_func(test_cfgs)-safety_margin > 0) * 2 - 1
test_labels = test_labels.reshape(test_preds.shape)
test_acc = torch.sum(test_preds == test_labels).item() / num_test
test_tpr = torch.sum(test_preds[test_labels == 1] == 1).item() / torch.sum(test_labels == 1).item()
test_tnr = torch.sum(test_preds[test_labels == -1] == -1).item() / torch.sum(test_labels == -1).item()
print(f'test_acc = {test_acc:.4f}')
print(f'test_tpr = {test_tpr:.4f}')
print(f'test_tnr = {test_tnr:.4f}')
print(f'num of support points = {len(fkdc.gains)}')

safety_margin = -0.7984
test_acc = 0.9625
test_tpr = 1.0000
test_tnr = 0.9604
num of support points = 467


### 3.2 Method II: Using higher-level collision_checker API
#### but still with the **given dataset**

In [7]:
print(train_labels)
print(torch.sum(train_labels == 1).item())
print(torch.sum(train_labels == -1).item())

tensor([-1., -1., -1.,  ..., -1., -1., -1.])
570
9430


In [8]:
import diffco as dc
import trimesh.transformations as tf

panda_urdf_robot = dc.FrankaPanda(
    simple_collision=False,
    load_gripper=True, 
    base_transform=torch.eye(4),
    device="cpu", load_visual_meshes=False)
print(panda_urdf_robot.joint_limits)
fkdc = dc.ForwardKinematicsDiffCo(
    robot=panda_urdf_robot,
    environment=None,
    gamma=10
)

# The default FrankaPanda model has 8 joints, including an extra joint for the gripper, ranging [0, 0.04]
aug_train_cfgs = torch.cat([train_cfgs, torch.ones(len(train_cfgs))[:, None] * 0.00], dim=1)
aug_test_cfgs = torch.cat([test_cfgs, torch.ones(len(test_cfgs))[:, None] * 0.00], dim=1)

fkdc.fit(aug_train_cfgs, train_labels, verbose=True, verify_ratio=0)
print(f'num of support points = {len(fkdc.perceptron.gains)}')
acc, tpr, tnr = fkdc.verify(aug_test_cfgs, test_labels, verbose=True)

Setting up allowed collision matrix with 100 random configurations


tensor([[-2.9671,  2.9671],
        [-1.8326,  1.8326],
        [-2.9671,  2.9671],
        [-3.1416,  0.0873],
        [-2.9671,  2.9671],
        [-0.0873,  3.8223],
        [-2.9671,  2.9671],
        [ 0.0000,  0.0400]])
Unique position link names: ['panda_link1', 'panda_link3', 'panda_link4', 'panda_link5', 'panda_link7', 'panda_link8', 'panda_leftfinger', 'panda_rightfinger', 'panda_virtual_ee_link']
DiffCo training...


  5% 461/10000 [00:00<00:06, 1437.37it/s]

Ended at iteration 461, cost 0.3246 secs
ACC: 1.0
DiffCo training done. 0.4302 secs cost
num of support points = 411
Positive labels: 106, Negative labels: 1894
Test acc: 0.9875, TPR 0.8491, TNR 0.9952
Biased Test acc: 0.9865, TPR 0.9245, TNR 0.9900


### 3.3 Method III: Using higher-level collision_checker API
#### but with an automatically generated dataset

In [9]:
import diffco as dc
import trimesh.transformations as tf

# Optionally, you can add obstacle information to the ForwardKinematicsDiffCo class
shape_env = dc.ShapeEnv(
    shapes={
        name: {
            'type': 'Cylinder', 
            'params': {'radius': rtb_shape.radius, 'height': rtb_shape.length},
            'transform': tf.translation_matrix(rtb_shape._wT[:3, 3])
        } for name, rtb_shape in cenv.objects.items()
    }
)
panda_urdf_robot = dc.FrankaPanda(
    simple_collision=True,
    load_gripper=True, 
    base_transform=torch.eye(4),
    device="cpu", load_visual_meshes=False)
print(panda_urdf_robot.joint_limits)
fkdc = dc.ForwardKinematicsDiffCo(
    robot=panda_urdf_robot,
    environment=shape_env,
    gamma=10
)

acc, tpr, tnr = fkdc.fit(num_samples=12000, verbose=True, verify_ratio=1./6, fix_joints=[7], fix_joint_values=[0.04])
print(f'num of support points = {len(fkdc.perceptron.gains)}')

Setting up allowed collision matrix with 100 random configurations
tensor([[-2.8973,  2.8973],
        [-1.7628,  1.7628],
        [-2.8973,  2.8973],
        [-3.0718, -0.0698],
        [-2.8973,  2.8973],
        [-0.0175,  3.7525],
        [-2.8973,  2.8973],
        [ 0.0000,  0.0400]])
Unique position link names: ['panda_link1', 'panda_link3', 'panda_link4', 'panda_link5', 'panda_link7', 'panda_link8', 'panda_leftfinger', 'panda_rightfinger']
Generating labels...
Labels generated in 13.37s
Positive verify labels: 334, Negative verify labels: 1666
label_verify: tensor([ 1., -1., -1.,  ..., -1., -1., -1.])
DiffCo training...


 16% 1575/10000 [00:00<00:05, 1673.27it/s]


Ended at iteration 1575, cost 0.9463 secs
ACC: 1.0
DiffCo training done. 1.0592 secs cost
Positive labels: 334, Negative labels: 1666
Test acc: 0.9355, TPR 0.8084, TNR 0.9610
Biased Test acc: 0.8680, TPR 0.9551, TNR 0.8505
num of support points = 1374


## 4. Define Trajectory Optimization Problem

Now we choose two free configurations from our dataset and attempt to find a path between them with trajectory optimization. It is possible optimization does not yield a successful result because it may get stuck at a local minimum. So we recommend obtaining a valid path with a motion planning algorithm as the initial solution for optimization.

The following block gets a pair of (collision-free) start and end configuration

In [10]:
free_cfgs = cfgs[labels == -1]
two_idx = np.random.choice(len(free_cfgs), 2, replace=False)
start_cfg, target_cfg = free_cfgs[two_idx]

from spatialmath import SE3
Tr = cenv.robot.fkine(cenv.robot.qr)
start_point = np.array([0.5, -0.3, 0.3])
target_point = np.array([0.5, 0.3, 0.5])
Tstart = SE3.Rt(R=Tr.R, t=start_point)
Ttarget = SE3.Rt(R=Tr.R, t=target_point)

while True:
    start_cfg = cenv.robot.ikine_LM(Tstart, mask=np.array([1., 1., 1., 1, 1, 1]), q0=cenv.robot.qr).q
    start_cfg = torch.tensor(start_cfg, dtype=torch.float32)
    target_cfg = cenv.robot.ikine_LM(Ttarget, mask=np.array([1., 1., 1., 1, 1, 1]), q0=cenv.robot.qr).q
    target_cfg = torch.tensor(target_cfg, dtype=torch.float32)
    if not any(cenv.is_collision([start_cfg, target_cfg])):
        break
print(f'start_cfg = {start_cfg}, fk_t = {cenv.robot.fkine(start_cfg).t}')
print(f'target_cfg = {target_cfg}, fk_t = {cenv.robot.fkine(target_cfg).t}')
print(f'collision = {cenv.is_collision([start_cfg, target_cfg])}')


pybullet build time: Nov 28 2023 23:51:11


start_cfg = tensor([-0.1864,  0.0947, -0.3750, -2.0538,  0.1104,  2.2240,  0.1634]), fk_t = [ 0.49999999 -0.3         0.29999999]
target_cfg = tensor([ 0.1778,  0.0207,  0.3855, -1.6237, -0.0618,  1.7273,  1.3554]), fk_t = [0.5        0.29999999 0.5       ]
collision = [False, False]


In [11]:
# set up an environmet in your browser for easier visualization
# (otherwise, you will need to scroll up a lot to see the environment)
tmp_cenv = PandaThreeCylinderEnv(launch_args={'realtime': False, 'browser': 'google-chrome'})
tmp_cenv.robot.q = start_cfg.numpy()
tmp_cenv.env.step(0.01)

### 4.1 (Recommended) Obtain a valid path with OMPL

In [12]:
from scripts.motion_planner import MotionPlanner

def my_motion_cost(s1, s2):
    # print(f'motion cost: {s1}, {s2}')
    p_tensor = torch.stack([s1, s2])
    control_points = drobot.fkine(p_tensor)
    diff = (control_points[1:]-control_points[:-1]).pow(2).sum()
    return diff.item()
def valid_check_func(x):
    # print(f'valid check: {x}')
    if not isinstance(x, np.ndarray):
        x = np.array(x)
    return not cenv._single_collision(x)
# valid_check_func = lambda x: not cenv.is_collision(x)
mp = MotionPlanner(drobot, valid_check_func, my_motion_cost)
ompl_rec = mp.plan(start_cfg, target_cfg, {'maxtime': 10.0})
print(f'OMPL success?: {ompl_rec["success"]}')
ompl_path = torch.tensor(ompl_rec['solution'], dtype=torch.double)

planner: RRTConnect
Info:    RRTConnect: Space information setup was not yet called. Calling now.
Debug:   RRTConnect: Planner range detected to be 2.607407
Info:    RRTConnect: Starting planning with 1 states already in datastructure


         at line 101 in /home/yuheng/dependencies/ompl-1.5.2/src/ompl/base/src/Planner.cpp


Info:    RRTConnect: Created 13 states (5 start + 8 goal)
OMPL success?: True


In [13]:
import time
print(f'OMPL path length {len(ompl_path)}')
tmp_cenv.robot.q = start_cfg.numpy()
tmp_cenv.env.step(0.01)
time.sleep(3)
for cfg in ompl_path:
    tmp_cenv.robot.q = cfg.numpy()
    tmp_cenv.env.step(0.05)
    time.sleep(0.05)

OMPL path length 114


### 4.2 Trajectory Optimization with DiffCo

In [14]:
# Make sure joint limits align in the diffco robot model and the RTB robot model
print(f'diffco robot model joint limits: {drobot.limits}')
print(f'RTB robot joint limits: {cenv.robot.qlim}')

diffco robot model joint limits: tensor([[-2.8973,  2.8973],
        [-1.7628,  1.7628],
        [-2.8973,  2.8973],
        [-3.0718, -0.0698],
        [-2.8973,  2.8973],
        [-0.0175,  3.7525],
        [-2.8973,  2.8973]])
RTB robot joint limits: [[-2.8973 -1.7628 -2.8973 -3.0718 -2.8973 -0.0175 -2.8973]
 [ 2.8973  1.7628  2.8973 -0.0698  2.8973  3.7525  2.8973]]


In [15]:
# Set up some optimization options
from diffco import optim
import importlib
importlib.reload(optim)
from diffco.optim import givengrad_traj_optimize, adam_traj_optimize, trustconstr_traj_optimize

optim_options = {
    'N_WAYPOINTS': 20,
    'NUM_RE_TRIALS': 1,
    'MAXITER': 30, # 200 for adam, 50 for slsqp(givengrad), 30 for trust_constr
    'safety_margin': safety_margin * 2,
    'max_speed': 0.1, # max interval between adjacent configs for collision checks
    'seed': 1234,
    'history': False,
    'extra_optimizer_options': {
        'disp': True, # False to suppress output
        # 'iprint': 10, # make slsqp verbose
        'verbose': 3, # make trust_constr verbose
        # 'lr': 0.1, # learning rate for adam
    },
}
robot = model.PandaFK()
try:
    if ompl_path is not None:
        init_path = ompl_path.clone()
        def sparsify_path(path, n):
            assert n >= 2
            if len(path) <= n:
                return path
            indices = np.linspace(0, len(path)-1, n, dtype=int)
            return path[indices]
        init_path = sparsify_path(init_path, optim_options['N_WAYPOINTS'])

        optim_options['init_solution'] = init_path
except NameError:
    pass

#### Choose one optimizer from below 
(and maybe adjust options above accordingly)

In [16]:
# Fastest, lower success rate and path quality
# optim_rec = adam_traj_optimize(
#     robot, col_func, start_cfg, target_cfg, optim_options)

# Medium success rate, better convergence than Adam
# optim_rec = givengrad_traj_optimize(
#     robot, col_func, start_cfg, target_cfg, optim_options)

# (Recommended) Slowest, highest success rate and path quality
optim_rec = trustconstr_traj_optimize(
    robot, col_func, start_cfg, target_cfg, optim_options)

# print(f'optim_rec = {optim_rec}')

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |   1   |   0   | +1.3363e+00 | 1.00e+00 | 8.17e-01 | 8.13e+00 | 1.00e+00 |  1.00e-01   |   0   |


/home/yuheng/miniconda3/envs/diffco/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


|   2   |   2   |   1   | +1.2619e+00 | 7.00e+00 | 1.41e+00 | 1.49e+00 | 1.00e+00 |  1.00e-01   |   2   |
|   3   |   3   |   7   | +9.3828e-01 | 7.00e+00 | 5.78e-01 | 2.13e-01 | 1.00e+00 |  1.00e-01   |   4   |
|   4   |   4   |  14   | +8.0567e-01 | 7.00e+00 | 2.52e-01 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   4   |
|   5   |   5   |  17   | +7.0894e-01 | 7.00e+00 | 1.62e-01 | 2.26e-03 | 1.34e+00 |  1.00e-01   |   4   |
|   6   |   6   |  20   | +7.0894e-01 | 7.00e-01 | 1.62e-01 | 2.26e-03 | 1.34e+00 |  1.00e-01   |   4   |
|   7   |   7   |  24   | +7.0894e-01 | 7.00e-02 | 1.62e-01 | 2.26e-03 | 1.34e+00 |  1.00e-01   |   4   |
|   8   |   8   |  25   | +6.8719e-01 | 1.40e-01 | 1.53e-01 | 1.09e-02 | 1.34e+00 |  1.00e-01   |   2   |
|   9   |   9   |  26   | +6.6185e-01 | 1.40e-01 | 2.00e-01 | 2.04e-02 | 1.34e+00 |  1.00e-01   |   2   |
|  10   |  10   |  27   | +6.2387e-01 | 2.80e-01 | 1.51e-01 | 9.46e-02 | 1.34e+00 |  1.00e-01   |   2   |
|  11   |  11   |  29   | +6.2387e-01 | 1.40e-

In [17]:
# Compare path cost before and after optimization
qs = optim_options['init_solution']
fk_points = robot.fkine(qs)
init_cost = (fk_points[1:] - fk_points[:-1]).pow(2).sum()
print(f'init cost = {init_cost}')
qs = optim_rec['solution']
qs = torch.tensor(qs)
fk_points = robot.fkine(qs)
cost = (fk_points[1:] - fk_points[:-1]).pow(2).sum()
print(f'cost = {cost}')
assert not torch.allclose(qs, optim_options['init_solution'].to(qs.dtype))

init cost = 1.3363447475452177
cost = 0.5509647130966187


In [18]:
# Verify and visualize the optimized path.
# You may also modify the code to visualize the initial path before optimization.

from diffco.utils import dense_path
qs = dense_path(qs, 0.2)

in_collision = cenv.is_collision(qs)
proxy_in_collision = (col_func(qs) - safety_margin > 0).reshape(-1)
print(f'Any in_collision ?: {any(in_collision)}')
print(f'Any proxy collision ?: {any(proxy_in_collision)}')

print(f'qs.shape = {qs.shape}')
import time
tmp_cenv.robot.q = start_cfg.numpy()
tmp_cenv.env.step(0.01)
time.sleep(3)
for q in qs:
    tmp_cenv.robot.q = q.numpy()
    tmp_cenv.env.step(0.05)
    time.sleep(0.05)

Any in_collision ?: False
Any proxy collision ?: False
qs.shape = torch.Size([71, 7])


### 4.3 Trajectory Optimization with Geometrical Collision Checking
We then also try optimization with the geometrical collison checker that comes with the environment, which is not differentiable.

Note: this may take more than **30 minutes** to converge and sometimes may not even converge.

In [19]:
from diffco.optim import gradient_free_traj_optimize
optim_options = {
    'N_WAYPOINTS': 20,
    'NUM_RE_TRIALS': 1,
    'MAXITER': 10,
    'max_speed': 0.1,
    'max_dense_waypoints': 200,
    'seed': 1234,
    'history': False,
    'extra_optimizer_options': {
        'disp': True, # False to suppress output
        # 'iprint': 10, # make slsqp verbose
        'verbose': 3, # make trust_constr verbose
        # 'lr': 0.1, # learning rate for adam
    }
}
try:
    optim_options['init_solution'] = init_path
except NameError:
    print('No init path, starting with straight line')
robot = model.PandaFK()
# dist_func = lambda qs: torch.tensor(cenv.distance(qs.numpy()), dtype=torch.float32)
def gt_dist_func(qs):
    # print(f'gt_dist_func: qs.shape = {qs.shape}')
    return torch.tensor(cenv.distance(qs.numpy()), dtype=torch.float32)
optim_rec = gradient_free_traj_optimize(
    robot, gt_dist_func, start_cfg, target_cfg, optim_options)

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |  127  |   0   | +1.3363e+00 | 1.00e+00 | 9.05e-01 | 1.96e+00 | 1.00e+00 |  1.00e-01   |   0   |
|   2   |  254  |   2   | +1.3912e+00 | 7.00e+00 | 7.90e-01 | 1.27e+00 | 1.00e+00 |  1.00e-01   |   2   |
|   3   |  381  |  12   | +2.4018e+00 | 1.88e+01 | 6.75e-01 | 6.40e-01 | 1.82e+00 |  1.00e-01   |   4   |
|   4   |  508  |  17   | +1.7396e+00 | 1.88e+01 | 6.67e-01 | 5.85e-01 | 1.82e+00 |  1.00e-01   |   4   |
|   5   |  635  |  33   | +2.0387e+00 | 1.88e+01 | 6.62e-01 | 6.71e-01 | 3.97e+00 |  1.00e-01   |   4   |
|   6   |  762  |  45   | +2.0387e+00 | 1.88e+00 | 6.62e-01 | 6.71e-01 | 3.97e+00 |  1.00e-01   |   4   |
|   7   |  889  |  50   | +2.0387e+00 | 9.41e-01 | 6.62e-01 | 6.71e-01 | 3.97e+00 |  1.00e-01   |   2   |
|   8   | 1016  |  52   | +2.1204e+00 | 9.41e-

In [20]:
qs = optim_options['init_solution']
fk_points = robot.fkine(qs)
init_cost = (fk_points[1:] - fk_points[:-1]).pow(2).sum()
print(f'init cost = {init_cost}')
qs = optim_rec['solution']
qs = torch.tensor(qs)
fk_points = robot.fkine(qs)
cost = (fk_points[1:] - fk_points[:-1]).pow(2).sum()
print(f'cost = {cost}')
assert not torch.allclose(qs, optim_options['init_solution'].to(qs.dtype))

print(f'time = {optim_rec["time"]}')

from diffco.utils import dense_path
qs = dense_path(qs, 0.2)

in_collision = cenv.is_collision(qs)
proxy_in_collision = (col_func(qs) - safety_margin > 0).reshape(-1)
print(f'Any in_collision = {any(in_collision)}')
print(f'Any proxy collision = {any(proxy_in_collision)}')

print(f'qs.shape = {qs.shape}')
import time
for q in qs:
    tmp_cenv.robot.q = q.numpy()
    tmp_cenv.env.step(0.05)
    time.sleep(0.05)

init cost = 1.3363447475452177
cost = 1.697069764137268
time = 279.83578610420227
Any in_collision = True
Any proxy collision = True
qs.shape = torch.Size([82, 7])
